In [1]:
import numpy as np
import pandas as pd
from google.cloud import firestore  ## anaconda cmd에 pip install --upgrade google-api-python-client, pip install --upgrade google-cloud 하고 
from google.cloud import bigquery   ## pip instal --upgrade google-cloud-firestore랑 google-cloud-bigquery까지 해야한다.
from google.cloud.bigquery import magics
import tqdm

# python에서 firestore에 데이터를 조작하려면 pip install firebase-admin을 해야한다.
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os, os.path

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import xlearn as xl    # pip로 다운은 되는데 import를 못해온다. https://github.com/aksnzhy/xlearn/releases 에 가서 win버젼 whl을 다운한뒤 
                        # 다운된 디렉터리에서 아나콘다 프롬프트로 pip install <file name>.whl을 하면 된다.

# Firestore에서 미팅 소개글 가져오기

In [2]:
# 환경변수 설정
# os.environ['GOOGLE_APPLICATION_CREDENTIALS']= 'C:/Users/JY/Desktop/yjh/git/AI_capstone2/cloudfunctions/a2i2-7a738-f9846d1e9895.json'

# Cloud Firestore 인스턴스 초기화
cred = credentials.Certificate('C:/Users/JY/Desktop/yjh/git/AI_capstone2/cloudfunctions/a2i2-7a738-f9846d1e9895.json')
firebase_admin.initialize_app(cred)

# DB의 경로를 참조하여 필드 추가 또는 수정
db = firestore.client()

ref = db.collection(u'products')
docs = ref.stream()

for doc in docs :
    print(f'{doc.id} => {doc.to_dict()}')


W2TO2uQlQ3T2iEvxxom3 => {'Products': [{'meeting_description': '평범한 직장인을 위한 현실적인 자산관리와 투자 방법론\n평범한 직장인이 투자를 시작할 때 최소한 알아야할 것\n꼭알고 있어야 할 금융상품 및 투자 방법론\n구체적인 미래를 숫자에 근거해 준비하는 태도', 'price': 30000, 'name': '김다솔', 'mentor_description': '유한양행 2년(학술)', 'title': '직장인을 위한 자산관리 ', 'image': ['no image', 'no image2'], 'rating': 0.1, 'isNew': True, 'id': 'v3IzhvjVHKhmFK16OvWnHs8wOjm2'}, {'name': '김건무', 'price': 30000, 'mentor_description': '강원산업경제진흥원 및 이공계 공기업 기획분야 경력\nkt 모바일 상품권 협업\n삼성전자 4년 (제품기획)', 'meeting_description': '많은 일에서 욕심을 버리는 법\n한번에 한가지씩 일에 집중하는 것\n우선순위를 결정하는 법\n계획 세우는 법', 'title': '할 거면 확실히 하고, 못할거면 미련도 갖지 마세요', 'image': ['no image', 'no image2'], 'isNew': True, 'rating': 0.1, 'id': 'v3IzhvjVHKhmFK16OvWnHs8wOjm2'}, {'name': '김선생', 'rating': 0.1, 'mentor_description': '의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang 22년(데이터아키텍쳐, 기업전략, 센서개발, 디스플레이개발, 시스템엔지니어링)', 'title': '기업의 문화를 알아가는 법, 연구소 문화 소개', 'isNew': True, 'meeting_description': '의학 연구소 연구원의 문화\nDTI의 문화\n직장인 문화 이해하기', 'price': 3000

# Firestore에 조언 소개글 데이터 추가

In [3]:
# 데이터 가져오기
doc_data = pd.read_excel('C:/Users/JY/Desktop/yjh/4-2/ai융캡2/데이터/설문데이터정리_전처리.xlsx')
doc_data

,name,pseudo_name,career,title,description
0,김다솔,김O솔,유한양행 2년(학술),직장인을 위한 자산관리,평범한 직장인을 위한 현실적인 자산관리와 투자 방법론\n평범한 직장인이 투자를 시작...
1,김건무,김O무,강원산업경제진흥원 및 이공계 공기업 기획분야 경력\nkt 모바일 상품권 협업\n삼성...,"할 거면 확실히 하고, 못할거면 미련도 갖지 마세요",많은 일에서 욕심을 버리는 법\n한번에 한가지씩 일에 집중하는 것\n우선순위를 결정...
2,김선생,김O생,의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang ...,"기업의 문화를 알아가는 법, 연구소 문화 소개",의학 연구소 연구원의 문화\nDTI의 문화\n직장인 문화 이해하기
3,김선생,김O생,의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang ...,직장다니면서 대학원 준비하는 법,어떤 대학원을 선택할 것인가?\n어떤 마음가짐이 필요할까?\n비용과 시간을 확보하는...
4,강지훈,강O훈,산림조합중앙회 중부목재유통센터 15년(품질관리),성실과 겸손을 이길 화려한 기술은 없다,핑계대지 않는 법\n책임감을 기르는 법\n디테일하게 업무를 처리하는 법
5,장해민,장O민,이스타 항공 5년(서비스),승무원 재미나게 승진하는 법,"위급 승객, 진상 승객 대처 꿀팁 제시\n퀵턴 재미있게 보내는 법\n빠르게 승진하는..."
6,박정남 (은퇴자),박O남 (은퇴자),개인사업 20년 (사장),개인사업을 위한 조언,혼자서 사업을 시작하는 방법\n세무업무와 법무업무를 처리하는 방법
7,김종휘,김O휘,동서대학교 교직원 28년 (시설관리 업무),한 직장 오래 다니는 법,한 직장 오래 다니는 법
8,홍성식,홍O식,한국델파이 6년\n현대자동차 3년 \n인피니언반도체 21년 \nHK세미텍 1년 (엔...,회사 다니며 평생 커리어 찾기,본질적인 일의 의미와 내게 맞는 일을 찾는 법\n지속 가능한 나만의 커리어 만들어가...
9,홍성식,홍O식,한국델파이 6년\n현대자동차 3년 \n인피니언반도체 21년 \nHK세미텍 1년 (엔...,은퇴자금 설계,슬기로운 은퇴 설계로 행복한 노후 만드는 법\n나에게 필요한 은퇴자금 알아보기\n효...


In [40]:
doc_ref = db.collection(u'products').document(u'W2TO2uQlQ3T2iEvxxom3')


# Firestore에서 collection-document아래 새 field를 추가하려면 '새로운 필드명'을 써야 한다. 기존의 필드명을 쓰면 값을 업데이트(덮어쓰기)함으로 주의!
# 따라서 동일한 필드명으로 여러 필드를 만들고 싶으면 중첩된 필드를 사용하여 최상위 필드명만 바꿔가며 사용해야 한다.
# 또한 .set({데이터}, merge=True)로 merge 인자를 써야 덮어쓰기가 아닌 추가가 된다.
doc_array = []
for i in range(len(doc_data)):
    image_face = 'assets/images/face' + str(i)
    image_desc = 'assets/images/description' + str(i)
    data = {
            u'id' : 'v3IzhvjVHKhmFK16OvWnHs8wOjm2',
            u'image' : [ image_face, image_desc],
            u'isNew' : True,
            u'meeting_description' : doc_data.loc[i]['description'],
            u'mentor_description' : doc_data.loc[i]['career'],
            u'name' : doc_data.loc[i]['name'],
            u'price' : 30000,
            u'rating' : 0.1,
            u'title' : doc_data.loc[i]['title'],
        }
    doc_array.append(data)
    doc_ref.update({
        u'Products' : doc_array   # 배열을 생성하면 자동으로 index를 추가하기에
})

# Firestore에서 미팅 소개글 데이터 가져오기

In [10]:
# Firestore에서 get해올때마다 행, 열의 순서를 모두 random하게 가져온다..
meetings = db.collection(u'products').document(u'W2TO2uQlQ3T2iEvxxom3').get()
print(meetings.to_dict().keys())
print(meetings.to_dict()['Products'])

dict_keys(['Products'])
[{'image': ['no image', 'no image2'], 'price': 30000, 'id': 'v3IzhvjVHKhmFK16OvWnHs8wOjm2', 'name': '김다솔', 'isNew': True, 'meeting_description': '평범한 직장인을 위한 현실적인 자산관리와 투자 방법론\n평범한 직장인이 투자를 시작할 때 최소한 알아야할 것\n꼭알고 있어야 할 금융상품 및 투자 방법론\n구체적인 미래를 숫자에 근거해 준비하는 태도', 'title': '직장인을 위한 자산관리 ', 'mentor_description': '유한양행 2년(학술)', 'rating': 0.1}, {'image': ['no image', 'no image2'], 'rating': 0.1, 'title': '할 거면 확실히 하고, 못할거면 미련도 갖지 마세요', 'id': 'v3IzhvjVHKhmFK16OvWnHs8wOjm2', 'mentor_description': '강원산업경제진흥원 및 이공계 공기업 기획분야 경력\nkt 모바일 상품권 협업\n삼성전자 4년 (제품기획)', 'name': '김건무', 'isNew': True, 'price': 30000, 'meeting_description': '많은 일에서 욕심을 버리는 법\n한번에 한가지씩 일에 집중하는 것\n우선순위를 결정하는 법\n계획 세우는 법'}, {'isNew': True, 'title': '기업의 문화를 알아가는 법, 연구소 문화 소개', 'price': 30000, 'image': ['no image', 'no image2'], 'rating': 0.1, 'mentor_description': '의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang 22년(데이터아키텍쳐, 기업전략, 센서개발, 디스플레이개발, 시스템엔지니어링)', 'id': 'v3IzhvjVHKhmFK16OvWnHs8wOjm2', 'meeting_descr

In [13]:
meetings_df = pd.DataFrame(meetings.to_dict()['Products'], )
meetings_df

,image,price,id,name,isNew,meeting_description,title,mentor_description,rating
0,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,김다솔,True,평범한 직장인을 위한 현실적인 자산관리와 투자 방법론\n평범한 직장인이 투자를 시작...,직장인을 위한 자산관리,유한양행 2년(학술),0.1
1,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,김건무,True,많은 일에서 욕심을 버리는 법\n한번에 한가지씩 일에 집중하는 것\n우선순위를 결정...,"할 거면 확실히 하고, 못할거면 미련도 갖지 마세요",강원산업경제진흥원 및 이공계 공기업 기획분야 경력\nkt 모바일 상품권 협업\n삼성...,0.1
2,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,김선생,True,의학 연구소 연구원의 문화\nDTI의 문화\n직장인 문화 이해하기,"기업의 문화를 알아가는 법, 연구소 문화 소개",의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang ...,0.1
3,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,김선생,True,어떤 대학원을 선택할 것인가?\n어떤 마음가짐이 필요할까?\n비용과 시간을 확보하는...,직장다니면서 대학원 준비하는 법,의학 연구소 연구원 9년\nT-Mobile Corning Sony DTI Wang ...,0.1
4,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,강지훈,True,핑계대지 않는 법\n책임감을 기르는 법\n디테일하게 업무를 처리하는 법,성실과 겸손을 이길 화려한 기술은 없다,산림조합중앙회 중부목재유통센터 15년(품질관리),0.1
5,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,장해민,True,"위급 승객, 진상 승객 대처 꿀팁 제시\n퀵턴 재미있게 보내는 법\n빠르게 승진하는...",승무원 재미나게 승진하는 법,이스타 항공 5년(서비스),0.1
6,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,박정남 (은퇴자),True,혼자서 사업을 시작하는 방법\n세무업무와 법무업무를 처리하는 방법,개인사업을 위한 조언,개인사업 20년 (사장),0.1
7,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,김종휘,True,한 직장 오래 다니는 법,한 직장 오래 다니는 법,동서대학교 교직원 28년 (시설관리 업무),0.1
8,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,홍성식,True,본질적인 일의 의미와 내게 맞는 일을 찾는 법\n지속 가능한 나만의 커리어 만들어가...,회사 다니며 평생 커리어 찾기,한국델파이 6년\n현대자동차 3년 \n인피니언반도체 21년 \nHK세미텍 1년 (엔...,0.1
9,"[no image, no image2]",30000,v3IzhvjVHKhmFK16OvWnHs8wOjm2,홍성식,True,슬기로운 은퇴 설계로 행복한 노후 만드는 법\n나에게 필요한 은퇴자금 알아보기\n효...,은퇴자금 설계,한국델파이 6년\n현대자동차 3년 \n인피니언반도체 21년 \nHK세미텍 1년 (엔...,0.1


In [17]:
# min_df : 토큰의 최소 발생 횟수. 이 값보다 적게 발생하면 무시한다.
# ngram_range : 토큰으로 사용할 n-gram(단어의 시퀀스)의 범위를 결정
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
desc_mat = count_vect.fit_transform(meetings_df['meeting_description'])
print(desc_mat.shape)

(19, 442)


In [18]:
desc_sim = cosine_similarity(desc_mat, desc_mat)
desc_sim[:5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.02421797, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02159168, 0.05357997, 0.        ,
        0.        , 0.02696799, 0.01852169, 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.09007547, 0.        , 0.        , 0.        ,
        0.        , 0.04503773, 0.0594701 , 0.        , 0.        ,
        0.074

In [19]:
desc = pd.DataFrame(desc_sim)
desc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.000000,0.0,0.0,0.000000,0.000000,0.024218,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.021592,0.053580,0.000000,0.000000,0.026968,0.018522,0.000000
1,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.090075,0.0,0.000000,0.0,0.000000,0.045038,0.059470,0.000000,0.000000,0.074278,0.000000,0.076522,0.037139
4,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.067267,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055470,0.000000,0.000000
5,0.024218,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.028760,0.071368,0.000000,0.000000,0.000000,0.024671,0.000000
6,0.000000,0.0,0.0,0.090075,0.067267,0.000000,1.000000,0.0,0.000000,0.0,0.000000,0.117647,0.155347,0.000000,0.000000,0.194029,0.000000,0.199889,0.097014
7,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.020847,0.000000,0.000000,0.000000,0.000000,0.017883,0.000000
9,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
def find_sim_meetings(df, sim_matrix, title_name, top_n=5):
    
    #입력한 미팅의 index
    title_meeting = df[df['title'] == title_name]
    title_index = title_meeting.index.values
    print(title_index)
    
    # 입력한 미팅의 유사도 데이터 프레임 추가
    df['similarity'] = sim_matrix[title_index, :].reshape(-1,1)
    
    # 유사도 내림차순 정렬 후 상위 index 추출
    temp = df.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[: top_n ]
    
    return df.iloc[final_index]

In [21]:
similar_meetings = find_sim_meetings(meetings_df, desc_sim, '성실과 겸손을 이길 화려한 기술은 없다', 5)
similar_meetings[['title', 'name', 'similarity']]

[4]


,title,name,similarity
4,성실과 겸손을 이길 화려한 기술은 없다,강지훈,1.000000
6,개인사업을 위한 조언,박정남 (은퇴자),0.067267
16,두산중공업 15년차의 직장생활 노하우,오태섭,0.055470
0,직장인을 위한 자산관리,김다솔,0.000000
11,보건환경 전문가의 의사소통 노하우,여진형,0.000000


# BigQuery에서 활동 로그 가져오기

In [22]:
# 환경변수 설정
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= 'C:/Users/JY/Desktop/yjh/git/AI_capstone2/cloudfunctions/a2i2-7a738-f9846d1e9895.json'

In [23]:
%reload_ext google.cloud.bigquery    
# IPython용 bigquery magic

In [32]:
%%bigquery app_log
SELECT
    *
FROM `analytics_292285048.events_20211124`, `analytics_292285048.events_20211125` 
LIMIT 3

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.11s/rows]


In [33]:
app_log

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,user_ltv_1,device_1,geo_1,app_info_1,traffic_source_1,stream_id_1,platform_1,event_dimensions_1,ecommerce_1,items_1
0,20211124,1637748267481000,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",NaN,NaN,2,781982,None,bbfc94b12212b93197d2aeeb5ab67e63,...,None,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Asia', 'country': 'South Korea'...","{'id': 'com.example.a2i2', 'version': '1.0.0',...","{'name': '(direct)', 'medium': '(none)', 'sour...",3033277401,ANDROID,None,None,[]
1,20211124,1637748267481000,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",NaN,NaN,2,781982,None,bbfc94b12212b93197d2aeeb5ab67e63,...,None,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Asia', 'country': 'South Korea'...","{'id': 'com.example.a2i2', 'version': '1.0.0',...","{'name': '(direct)', 'medium': '(none)', 'sour...",3033277401,ANDROID,None,None,[]
2,20211124,1637748267481000,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",NaN,NaN,2,781982,None,bbfc94b12212b93197d2aeeb5ab67e63,...,None,"{'category': 'mobile', 'mobile_brand_name': 'S...","{'continent': 'Asia', 'country': 'South Korea'...","{'id': 'com.example.a2i2', 'version': '1.0.0',...","{'name': '(direct)', 'medium': '(none)', 'sour...",3033277401,ANDROID,None,None,[]


In [31]:
xl.hello()

In [7]:
def main(request):
    pass
